In [2]:
import os
from os.path import exists, join, split
from glob import glob
import numpy as np
import _pickle as pkl
from psbody.mesh import Mesh, MeshViewer, MeshViewers

from MultiGarmentNetwork.utils.smpl_paths import SmplPaths
from MultiGarmentNetwork.lib.ch_smpl import Smpl
from MultiGarmentNetwork.dress_SMPL import load_smpl_from_file, pose_garment
from MultiGarmentNetwork.utils.interpenetration_ind import remove_interpenetration_fast

from IPython.display import Image

In [3]:
path = '../Multi-Garment_dataset/'
# all_scans = glob(path + '*')
garment_classes = ['Pants', 'ShortPants', 'ShirtNoCoat', 'TShirtNoCoat', 'LongCoat']
gar_dict = {}

for gar in garment_classes:
    gar_dict[gar] = glob(join(path, '*', gar + '.obj'))

In [4]:
# dp = SmplPaths()
# vt, ft = dp.get_vt_ft_hres()
# smpl = Smpl(dp.get_hres_smpl_model_data())

In [5]:
## This file contains correspondances between garment vertices and smpl body
fts_file = 'MultiGarmentNetwork/assets/garment_fts.pkl'
vert_indices, fts = pkl.load(open(fts_file, 'rb') , encoding='latin1')
# fts['naked'] = ft

In [6]:
## Choose any garment type
garment_type = 'Pants'
index = np.random.randint(0, len(gar_dict[garment_type]))   ## Randomly pick from the digital wardrobe

In [7]:
with_tex = True
garment_path = gar_dict[garment_type][index]

In [8]:
garment_path

'../Multi-Garment_dataset/125611521914479/Pants.obj'

In [19]:
## Load SMPL body for the garment
path = split(garment_path)[0]
garment_org_body = load_smpl_from_file(join(path, 'registration.pkl'))
garment_org_body = Mesh(garment_org_body.v, garment_org_body.f)

In [16]:
## Load unposed garment
garment_unposed = Mesh(filename=garment_path)
garment_unposed.set_texture_image(join(path, 'multi_tex.jpg'))

In [17]:
## Pose garments
dat = pkl.load(open(join(path, 'registration.pkl'), 'rb') , encoding='latin1')
dat['gender'] = 'neutral'
garment_posed = pose_garment(garment_unposed, vert_indices[garment_type], dat)
garment_posed = remove_interpenetration_fast(garment_posed, garment_org_body)

In [18]:
garment_posed

In [13]:
if with_tex:
    garment_posed.vt = garment_unposed.vt
    garment_posed.ft = garment_unposed.ft
    garment_posed.set_texture_image(join(path, 'multi_tex.jpg'))

In [20]:
mvs = MeshViewers((1, 3), keepalive=True)

mvs[0][2].set_background_color(np.array([1,1,1]))
mvs[0][1].set_background_color(np.array([1,1,1]))
mvs[0][0].set_background_color(np.array([1,1,1]))

In [21]:
mvs

In [22]:
mvs[0][1].set_static_meshes([garment_org_body])
mvs[0][1].save_snapshot('garment_org_body.png', blocking=True)

In [23]:
mvs[0][2].set_static_meshes([garment_org_body, garment_posed])
mvs[0][2].save_snapshot('garment_org_body_garment_posed.png', blocking=True)

In [24]:
mvs[0][0].set_static_meshes([garment_unposed])
mvs[0][0].save_snapshot('garment_unposed.png', blocking=True)

In [46]:
# Image('garment_org_body.png')